In [69]:
import os.path
import sys
from os import listdir

def tableName(cursor, run='004'):
    tbName0 = 'rct007_testID'
    tbName1 = run+'_flowindata'
    tbName2 = run+'_flowoutdata'
    tbName3 = run+'_plcdata'
    tbName4 = run+'_voltagedata'
    tbName5 = run+'_logdata'
    return [tbName0, tbName1, tbName2, tbName3, tbName4, tbName5]

def parseLog(cursor, run):
    # read log file
    log = readNoDateTable(cursor, run+'_logdata')
    
    # get the start and stop states by looking at state variable
    stateStart = log[log.state=='1'].date.reset_index(drop=True)
    stateEnd = log[log.state=='0'].date.reset_index(drop=True) 
    
    # check to ensure that we have the same number of states
    lenStart = len(stateStart)
    lenEnd = len(stateEnd)
    if lenStart != lenEnd:
        stop = min(lenStart, lenEnd)
        stateStart = stateStart[0:stop]
        stateEnd = stateEnd[0:stop]
        print 'States have been truncated start:%d end:%d'%(lenStart, lenEnd)
    
    return stateStart, stateEnd

def getEventIdOffset(cursor, run):
    initialRun = 4
    runNum = int(run)
    offset = 0
    if runNum>initialRun:
        for ii in np.arange(initialRun, runNum):
            [s,e] = parseLog(cursor, str(ii).zfill(3))
            offset += len(s)
            print ii, len(s), offset
    return offset

def pickleTable(df, path, dbase, run, eventID):
    # get rid of date
    df.drop('date', axis=1, inplace=True)
    
    #all the files to read and save
    for col in df.columns:
        filename = path+dbase+'_'+run+'_'+col+'_'+str(eventID)+'.pkl'
        print 'Pickling %s \n'%filename
        pd.DataFrame(df[col]).to_pickle(filename)

def pickleEvent(dbase, run, start, end, event, path):
    cursor = connectDbase(dbase)

    # get table names
    [t0,t1,t2,t3,t4,t5] = tableName(run)
            
    fin = readDateTable(cursor, t1, start, end)
    pickleTable(fin, path, dbase, run, event)

    fout = readDateTable(cursor, t2, start, end)
    pickleTable(fout, path, dbase, run, event)
    
    plc = readDateTable(cursor, t3, start, end)
    pickleTable(plc, path, dbase, run, event)

    volt = readDateTable(cursor, t4, start, end)
    pickleTable(volt, path, dbase, run, event)
    
def pickleSensorFiles(cursor, dbase, run, path, forceWrite = False):

    [start, e] = parseLog(cursor, run)
    offset = getEventIdOffset(cursor, run)

    # find the last event id stored to pickle
    dirList = os.listdir(path)
    pklFileList = [s for s in dirList if 'pkl' in s and not '_log.pkl' in s]
    temp = [x.split('_')[-1] for x in pklFileList]    
    storedEvent = max([int(x.split('.')[0]) for x in temp])
  
    for ii,s in enumerate(start):
        # only do this for new event ids
        if forceWrite or (ii>storedEvent+offset):
            pickleEvent(dbase, run, s, e[ii], ii+offset, path)

def logFileDataFrame(cursor, dbase, run):           
    # read and store log file
    log = readNoDateTable(cursor, run+'_logdata')

    # error check and format log file
    # keep only states 1 and 0
    log = log[(log['state'] == '1') | (log['state'] == '0')].reset_index(drop=True)

    # if the comment contains the word 'error' get rid of that row
    idx  = ['error' not in c.lower() for c in log.comment.values]
    log = log[idx].reset_index(drop=True)

    # remove repeated states
    # here we keep the first state remove the rest that repeat
    states = log.state.apply(int)
    dState = states.diff()
    dState[0] = 1
    log = log[(dState == 1) | (dState == -1)].reset_index(drop=True)

    # if last state is not STOP, then get rid of it
    lastState = int(log.iloc[-1,:]['state'])
    
    if lastState == 1:
        log = log.iloc[0:-1,:].reset_index(drop=True)

    # check time stamps.
    if log.date.diff().dt.seconds.any() < 0:
         sys.exit("Non-Increasing Sequential Time in events")

    # add event ids and run
    eventId = np.arange(len(log[log.state == '1'])).repeat(2)
    log['eventId'] = eventId
    log['run'] = run

    return log
    
def pickleLogFile(cursor, dbase, runList, path, forceWrite = False):
    
    # init variables 
    filename = path+dbase+'_'+'log'+'.pkl'
    dfList = []
    
    # get log data frame
    for runNum in runList:
        dfList.append(logFileDataFrame(cursor, dbase, runNum))
        
    # make log file from list of dataframes
    dfLog = pd.concat(dfList, axis=0, ignore_index=True)
        
    # only if we have not done so already
    if forceWrite or not os.path.isfile(filename):
        dfLog.to_pickle(filename)
        


In [ ]:
from IPython.display import HTML

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="th", props=[("font-size", "100%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]
# html1 = (valve.style.set_table_styles(styles))
# html2 = (rctTemp.style.set_table_styles(styles))


In [ ]:
import sys
import time
from IPython.display import display,HTML,clear_output
def dashboard(run='002', updateSec=3):

    cursor = connectDbase('rct007')

    tbName0 = 'rct007_testID'
    tbName1 = run+'_flowindata'
    tbName2 = run+'_flowoutdata'
    tbName3 = run+'_plcdata'
    tbName4 = run+'_voltagedata'

    while(True):
        time.sleep(updateSec)

        cursor = connectDbase('rct007')
        plc   = readLastNTable(cursor, tbName3,1)
        volt  = readLastNTable(cursor, tbName4,1)
        flin  = readLastNTable(cursor, tbName1)
        flout = readLastNTable(cursor, tbName2)

        valve = plc.loc[:,['foreline_valve_closed', 'hivac_valve_closed','rough_valve_closed','sys_rct_valve_closed','sys_rga_valve_closed']]
        valve.rename(columns={'foreline_valve_closed':'Foreline', 'hivac_valve_closed':'Hivac','rough_valve_closed':'Rough','sys_rct_valve_closed':'Sys RCT','sys_rga_valve_closed':'Sys RGA'}, inplace=True)
        vhtml = (valve.style.set_table_styles(styles)).set_caption('Valve State')


        rctTemp = plc.loc[:, ['rct_rtd1', 'rct_rtd2', 'rct_rtd3', 'rct_rtd4', 'rct_rtd5', 'rct_rtd6']]
        rctThtml = (rctTemp.style.set_table_styles(styles)).set_caption('Reactor Temperatures')

        ambTemp = plc.loc[:, ['amb_rtd1', 'amb_rtd2', 'amb_rtd3', 'amb_rtd4']]
        ambThtml = (ambTemp.style.set_table_styles(styles)).set_caption('Ambient Temperatures')

        press = plc.loc[:, ['BaratronPress', 'forelinePress', 'systemPress']]
        press.rename(columns={'BaratronPress':'Baratron', 'forelinePress':'Foreline','systemPress':'System'}, inplace=True)                    
        phtml = (press.style.set_table_styles(styles)).set_caption('Pressure Readings')

        heat = plc.loc[:, ['htrstate', 'htrtimeon', 'htrperiod', 'htr1duty', 'htr2duty','htr1_tc','htr2_tc']]
        heat.rename(columns={'htrperiod':'Period', 'htrstate':'State', 'htr1duty':'Duty Cycle H1','htr2duty':'Duty Cycle H2','htrtimeon':'Time On','htr1_tc':'Temp H1','htr2_tc':'Temp H2'}, inplace=True)                    
        hhtml = (heat.style.set_table_styles(styles)).set_caption('Heater Settings')

        flow = plc.loc[:,['flowin_rtd','flowout_rtd'] ]
        flow['rate in'] = flin.flowin[0]
        flow['rate out'] = flout.flowout[0]
        fhtml = (flow.style.set_table_styles(styles).set_caption('Water Cooler Flow'))

        clear_output(wait=True)

        #display(HTML('<font size=4><b>Foreline Pressure %3f</b></font>'%plc.forelinePress[0]))
        display(HTML('<font size=4><b>Kernel -> Interrupt To Stop</b></font>'))
        display((vhtml))
        display((phtml))
        display(rctThtml)
        display(hhtml)
        display(fhtml)